In [1]:
import io
from datetime import date, datetime
from ftplib import FTP

import pandas as pd
from config import Config

In [2]:
ftp_server = Config.FTP_SERVER
ftp_username = Config.FTP_USERNAME
ftp_password = Config.FTP_PASSWORD
current_date = date.today().strftime("%Y_%m_%d")

In [3]:
ftp = FTP(ftp_server)
ftp.login(ftp_username, ftp_password)
ftp.cwd("group_report/")

'250 CWD command successful.'

In [4]:
csv_files = [x for x in ftp.nlst() if ("UAE" in x or "OM" in x) and current_date in x]

In [5]:
csv_files

['OM_Marginreport_GWG_2023_12_29_080013.csv',
 'OM_Marginreport_GWG_BLANK_2023_12_29_080013.csv',
 'UAE_Marginreport_GWG_2023_12_29_080013.csv',
 'UAE_Marginreport_GWG_BLANK_2023_12_29_080013.csv',
 'YT_OM_Marginreport_GWG_2023_12_29_080302.csv',
 'YT_UAE_Marginreport_GWG_2023_12_29_055010.csv']

In [6]:
if csv_files:
    df_set = []
    for csv in csv_files:
        cols = [0, 1, 3] if csv.startswith("YT") else [0, 1, 5]
        data = io.BytesIO()
        data.seek(0)
        ftp.retrbinary("RETR " + csv, data.write)
        data.seek(0)
        df = pd.read_csv(
            data,
            sep=";",
            usecols=cols,
            header=None,
            names=["bkg_ref", "operator_code", "operator_price"],
            thousands=",",
            dtype={"bkg_ref": str, "operator_code": str, "operator_price": float},
        )

        df.dropna(inplace=True)
        df_set.append(df)

    operator_cost = pd.concat(df_set, axis=0, ignore_index=True)

In [7]:
operator_cost.to_csv("operator_cost.csv", index=False)